In [1]:
import os
import sagemaker
from sagemaker.session import s3_input

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [6]:
inputs = sagemaker_session.upload_data(path='data')
display(inputs)

's3://sagemaker-us-west-2-453691756499/data'

In [2]:
hyperparams={'epochs': 20,
                 'learning-rate': 0.01,
                 'batch-size': 256,
                 'weight-decay': 2e-4,
                 'momentum': 0.9,
                 'optimizer': 'sgd'}

In [3]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='cifar10-training-script.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local_gpu',
                          framework_version='1.13', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparams)

In [8]:
train_path = 's3://sagemaker-container-webinar/cifar10-dataset/train'
val_path = 's3://sagemaker-container-webinar/cifar10-dataset/validation'
eval_path = 's3://sagemaker-container-webinar/cifar10-dataset/eval'

tf_estimator.fit({'training': train_path,'validation': val_path,'eval': eval_path})

Creating tmpcasd1150_algo-1-vwkzb_1 ... 
Attaching to tmpcasd1150_algo-1-vwkzb_12mdone
algo-1-vwkzb_1  | 2019-08-04 10:34:28,217 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-vwkzb_1  | 2019-08-04 10:34:28,780 sagemaker-containers INFO     Invoking user script
algo-1-vwkzb_1  | 
algo-1-vwkzb_1  | Training Env:
algo-1-vwkzb_1  | 
algo-1-vwkzb_1  | {
algo-1-vwkzb_1  |     "additional_framework_parameters": {},
algo-1-vwkzb_1  |     "channel_input_dirs": {
algo-1-vwkzb_1  |         "training": "/opt/ml/input/data/training",
algo-1-vwkzb_1  |         "validation": "/opt/ml/input/data/validation",
algo-1-vwkzb_1  |         "eval": "/opt/ml/input/data/eval"
algo-1-vwkzb_1  |     },
algo-1-vwkzb_1  |     "current_host": "algo-1-vwkzb",
algo-1-vwkzb_1  |     "framework_module": "sagemaker_tensorflow_container.training:main",
algo-1-vwkzb_1  |     "hosts": [
algo-1-vwkzb_1  |         "algo-1-vwkzb"
algo-1-vwkzb_1  |     ],
algo-1-vwkzb_1  |     

KeyboardInterrupt: 

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'epochs':        IntegerParameter(20, 100),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
    'batch-size':    IntegerParameter(32, 1024),
    'dense-layer':   IntegerParameter(128, 1024),
    'dropout':       ContinuousParameter(0.2, 0.6)
}

objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=10,
                            max_parallel_jobs=2,
                            objective_type=objective_type)